<a href="https://colab.research.google.com/github/ashhdubey/Project---Movie-Recommendation-System/blob/main/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Movie Recommendation System

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# Load the dataset

In [2]:
url = 'https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Movies%20Recommendation.csv'
df = pd.read_csv(url)

# Select relevant features for recommendation

In [3]:
df_features = df[['Movie_Genre', 'Movie_Keywords', 'Movie_Tagline', 'Movie_Cast', 'Movie_Director']].fillna('')

# Combine textual features into a single column

In [4]:
df_features['combined_features'] = df_features['Movie_Genre'] + ' ' + df_features['Movie_Keywords'] + ' ' + \
                                   df_features['Movie_Tagline'] + ' ' + df_features['Movie_Cast'] + ' ' + \
                                   df_features['Movie_Director']

# TF-IDF Vectorization

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(df_features['combined_features'])

# Compute cosine similarity matrix

In [6]:
cosine_sim = cosine_similarity(X, X)

# Function to recommend movies

In [7]:
def recommend_movies(movie_name, cosine_sim, df, top=10):
    indices = pd.Series(df.index, index=df['Movie_Title']).drop_duplicates()
    try:
        idx = indices[movie_name]
    except KeyError:
        close_matches = difflib.get_close_matches(movie_name, df['Movie_Title'])
        if not close_matches:
            print("No close match found for input movie name.")
            return
        else:
            print(f"Did you mean '{close_matches[0]}'?")
            idx = indices[close_matches[0]]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top+1]
    movie_indices = [i[0] for i in sim_scores]

    return df['Movie_Title'].iloc[movie_indices]

# Example usage: Ask user for favorite movie and recommend similar movies

In [ ]:
favorite_movie_name = input('Enter your favorite movie name: ')

recommended_movies = recommend_movies(favorite_movie_name, cosine_sim, df)
if recommended_movies is not None:
    print(f'Top 10 Movies suggested for you based on "{favorite_movie_name}":')
    print(recommended_movies)
